In [3]:
#%%writefile OI_Data_V1.py
#importing Imp Libraries 

import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import datetime, time, math

start_time = time.time()
#df4 for final storing all values
df4 = pd.DataFrame(columns=['Date_Time','Strike_price','SUM_Call_OI_11', 'SUM_Put_OI_11', 'Difference',\
                                 'PCR_OI_11','PCR_Vol_11','PCR_OI_Total','PCR_Vol_Total'])
#---------------While loop for every 3 min -------#
j=0
while j<=5:
    # Create a session and load the page
    chromedriver_path = 'chromedriver.exe' # Change it to your own chromedriver path!
    driver = webdriver.Chrome(executable_path=chromedriver_path)
    driver.get('https://www1.nseindia.com/live_market/dynaContent/live_watch/option_chain/optionKeys.jsp')

    # Wait for the page to fully load
    driver.implicitly_wait(5)

    # Parse HTML code and grab tables with Beautiful Soup
    soup = BeautifulSoup(driver.page_source, 'lxml')

    tables = soup.find_all('table')

    # Read tables with Pandas read_html()
    dfs = pd.read_html(str(tables))

    #print(f'Total tables: {len(dfs)}')
    #print(dfs[0])

    #create an empty df
    df = pd.DataFrame() 
    df=dfs[1]

    #df.tail()

    df.columns = df.columns.droplevel() #it drops multilevel indexing

    #df.columns

    #drop unneccessary columns from df
    df1 = df.iloc[:, [1,2,3,11,19,20,21]]
    #df1.head()

    df1.columns = ['Call_OI', 'Call_chang_OI','Call_volume','Strike_price', 'Put_volume','Put_chang_OI','Put_OI']

    #df1.tail()

    total_df = df1.iloc[-1:]
    #total_df.head()

    Total_call_OI = int(total_df['Call_OI'])
    #print('Total PUT OI: ',Total_call_OI)

    Total_call_volume = int(total_df['Call_volume'])
    #print('Total Call Volume: ',Total_call_volume)

    Total_put_OI = int(total_df['Put_OI'])
    #print('Total PUT OI: ',Total_put_OI)

    Total_put_volume =int(total_df['Put_volume'])
    #print('Total Put Volume: ',Total_put_volume)

    #getting strick price from web page
    strick_price = driver.find_element_by_xpath('//*[@id="wrapper_btm"]/table[1]/tbody/tr/td[2]/div/span[1]/b')
    #print(strick_price.text)

    LTP=(strick_price.text[6:])
    LTP.strip()
    LTP=int(float(LTP))
    #print(LTP)

    #logic for strick price conversion of multiple of 150 -50
    f3digit=LTP//100
    #print("First 3 digit",f3digit)

    num=LTP//10
    last=num%10
    #print("Last: ",last)
    if last>=5:
        f3digit=(f3digit+1)*100
    else:
        f3digit=(f3digit)*100
        f3digit+=50

    #print(f3digit)

    #it gives index of matched column value in int form
    i=df1.loc[df1['Strike_price']==f3digit].index.values.astype(int)[0]
    #i

    start=i-5
    df2=df1.iloc[start:start+11,]

    #df2.head(11)

    #saving df into csv
    date_time=str(datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S'))
    name='ST_OI_NSE'+date_time+'.csv'
    df2.to_csv(name) 

    try:
        call_OI_11=(df2['Call_OI'].astype(str).astype(int)).sum()
        #print('CALL_OI_11 ',call_OI_11)

        put_OI_11=(df2['Put_OI'].astype(str).astype(int)).sum()
        #print('PUT_OI_11 ',put_OI_11)

        call_volume_11 =(df2['Call_volume'].astype(str).astype(int)).sum()
        #print('CALL_Volume_11',call_volume_11)

        put_volume_11 =(df2['Put_volume'].astype(str).astype(int)).sum()
        #print('PUT_Volume_11',put_volume_11) 

    except Exception as e:
        call_OI_11=0
        print('Ecxeption')
    difference = put_OI_11-call_OI_11
    #print('Difference: ',difference)

    PCR_OI_11=put_OI_11/call_OI_11

    PCR_OI_Total=Total_put_volume/Total_call_volume

    PCR_Vol_11=put_volume_11/call_volume_11

    PCR_Vol_Total=Total_put_volume/Total_call_volume



    #creating a data frame to store Date_Time, Sum of Call_IO, Sum of Put_IO, Difference and PCR


    df4 = df4.append({'Date_Time':date_time,\
                      'Strike_price':f3digit,\
                      'SUM_Call_OI_11': call_OI_11,\
                      'SUM_Put_OI_11': put_OI_11,\
                      'Difference': difference,\
                      'PCR_OI_11': PCR_OI_11,\
                      'PCR_Vol_11': PCR_Vol_11,\
                      'PCR_OI_Total': PCR_OI_11,\
                      'PCR_Vol_Total': PCR_Vol_Total},\
                     ignore_index=True)
    
    time.sleep(600)
    #print(df4.head())
    driver.close()
    print(j)
    j+=1
    print(df4.head())
#saving finaL df4 into csv
date_time=str(datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S'))
name='Final_OI_NSE'+date_time+'.csv'
df4.to_csv(name) 

0
1
2


TimeoutException: Message: timeout: Timed out receiving message from renderer: 298.668
  (Session info: chrome=87.0.4280.88)


Date_Time Strike_price SUM_Call_OI_11 SUM_Put_OI_11 Difference  \
0  2020-12-28 10-48-46        13850       18753450      11340750   -7412700   
1  2020-12-28 10-59-00        13850       19025475      11646075   -7379400   
2  2020-12-28 11-09-07        13850       19189425      11781375   -7408050   

   PCR_OI_11  PCR_Vol_11  PCR_OI_Total  PCR_Vol_Total  
0   0.604729    0.705354      0.604729       1.028511  
1   0.612131    0.711972      0.612131       1.025256  
2   0.613951    0.714648      0.613951       1.028750

https://medium.com/@thabo_65610/three-ways-to-automate-python-via-jupyter-notebook-d14aaa78de9

#Sum of All Row
try:
    Total_call_OI = (df1['Call_OI'].astype(str).astype(int)).sum()
    print('Total Call OI: ',Total_call_OI)
    
    Total_call_volume =(df1['Call_volume'].astype(str).astype(int)).sum()
    print('Total Call Volume: ',Total_call_volume)
    
    Total_put_OI = (df1['Put_OI'].astype(str).astype(int)).sum()
    print('Total PUT OI: ',Total_put_OI)
    
    Total_put_volume =(df1['Put_volume'].astype(str).astype(int)).sum()
    print('Total Put Volume: ',Total_put_volume)
except Exception as e:
    c_col=0
    print('Ecxeption')
